In [1]:
import sys
sys.path.append("./reference")
import torch as t
from reference.glide_text2im.download import load_checkpoint
from reference.glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)

device = t.device('cpu')
options = model_and_diffusion_defaults()
options['timestep_respacing'] = '100'
ref_model, diffusion = create_model_and_diffusion(**options)
ref_model.load_state_dict(load_checkpoint('base', device))


<All keys matched successfully>

In [23]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from unet import Text2Im
from reference.glide_text2im.tokenizer.bpe import get_encoder

In [28]:
tokenizer = get_encoder()
model = Text2Im(tokenizer)
# model

[768, 768, 768, 576, 576, 576, 576, 384, 384, 384, 384, 192, 192, 192, 192, 192]
1536
1536
1536
1344
1152
1152
1152
960
768
768
768
576
384
384
384
384


In [29]:
print(len(model.state_dict().keys()), len(ref_model.state_dict().keys()))
list(zip(model.state_dict().keys(), ref_model.state_dict().keys()))

783 783


[('positional_embedding', 'positional_embedding'),
 ('padding_embedding', 'padding_embedding'),
 ('time_embed.0.weight', 'time_embed.0.weight'),
 ('time_embed.0.bias', 'time_embed.0.bias'),
 ('time_embed.2.weight', 'time_embed.2.weight'),
 ('time_embed.2.bias', 'time_embed.2.bias'),
 ('in_layers.0.weight', 'input_blocks.0.0.weight'),
 ('in_layers.0.bias', 'input_blocks.0.0.bias'),
 ('in_layers.1.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.weight'),
 ('in_layers.1.in_layers.0.bias', 'input_blocks.1.0.in_layers.0.bias'),
 ('in_layers.1.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.weight'),
 ('in_layers.1.in_layers.2.bias', 'input_blocks.1.0.in_layers.2.bias'),
 ('in_layers.1.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.weight'),
 ('in_layers.1.emb_layers.1.bias', 'input_blocks.1.0.emb_layers.1.bias'),
 ('in_layers.1.group_norm.weight', 'input_blocks.1.0.out_layers.0.weight'),
 ('in_layers.1.group_norm.bias', 'input_blocks.1.0.out_layers.0.bias'),
 ('in_layers.1.out_l

In [30]:
state = ref_model.state_dict()
mapped = {ours: state[theirs] for ours, theirs in zip(model.state_dict().keys(), ref_model.state_dict().keys())}

model.load_state_dict(mapped)

RuntimeError: Error(s) in loading state_dict for Text2Im:
	size mismatch for out_layers.3.in_layers.2.weight: copying a param with shape torch.Size([768, 1344, 3, 3]) from checkpoint, the shape in current model is torch.Size([576, 1344, 3, 3]).
	size mismatch for out_layers.3.in_layers.2.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([576]).
	size mismatch for out_layers.3.emb_layers.1.weight: copying a param with shape torch.Size([1536, 768]) from checkpoint, the shape in current model is torch.Size([1152, 768]).
	size mismatch for out_layers.3.emb_layers.1.bias: copying a param with shape torch.Size([1536]) from checkpoint, the shape in current model is torch.Size([1152]).
	size mismatch for out_layers.3.group_norm.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([576]).
	size mismatch for out_layers.3.group_norm.bias: copying a param with shape torch.Size([768]